### Import all necessary modules

In [ ]:
import numpy as np
import sep

# additional setup for reading the test image and displaying plots
from astropy.io import fits
import matplotlib.pyplot as plt
from matplotlib import rcParams

%matplotlib inline

rcParams['figure.figsize'] = [10., 8.]

### Read example image from FITS file 

In [ ]:
# read image into standard 2-d numpy array
img = fits.open('data/image.fits')
data = img[0].data

# show the image
m, s = np.mean(data), np.std(data)
plt.imshow(data, interpolation='nearest', cmap='gray', vmin=m-s, vmax=m+s, origin='lower')
plt.colorbar()

# Save image
plt.savefig('tutorial-1.png')

### Background subtraction

In [ ]:
# measure a spatially varying background on the image
bkg = sep.Background(data)

# get a "global" mean and noise of the image background:
print(bkg.globalback)
print(bkg.globalrms)

In [ ]:
# evaluate background as 2-d array, same size as original image
bkg_image = bkg.back()

# show the background
plt.imshow(bkg_image, interpolation='nearest', cmap='gray', origin='lower')
plt.colorbar()

# Save image
plt.savefig('tutorial-2.png')

In [ ]:
# evaluate the background noise as 2-d array, same size as original image
bkg_rms = bkg.rms()

# show the background noise
plt.imshow(bkg_rms, interpolation='nearest', cmap='gray', origin='lower')
plt.colorbar()

# Save image
plt.savefig('tutorial-3.png')

In [ ]:
# Subtract the background.
data_sub = data - bkg

### Object detection

In [ ]:
# "setting the detection threshold to be a constant value of 1.5σ where σ is the global background RMS."
objects = sep.extract(data_sub, 1.5, err=bkg.globalrms)

# how many objects were detected.
len(objects)

In [ ]:
from matplotlib.patches import Ellipse

# plot background-subtracted image.
fig, ax = plt.subplots()
m, s = np.mean(data_sub), np.std(data_sub)
im = ax.imshow(data_sub, interpolation='nearest', cmap='gray', 
               vmin=m-s, vmax=m+s, origin='lower')

# plot an ellipse for each object.
for i in range(len(objects)):
    e = Ellipse(xy=(objects['x'][i], objects['y'][i]),
               width=6 * objects['a'][i],
               height=6 * objects['b'][i],
               angle=objects['theta'][i] * 180. / np.pi)
    e.set_facecolor('none')
    e.set_edgecolor('red')
    ax.add_artist(e)
    
# Save the image as PNG.
plt.savefig('tutorial-4.png')

In [ ]:
# Available fields.
objects.dtype.names

### Aperture Photometry

In [ ]:
flux, fluxerr, flag = sep.sum_circle(data_sub, objects['x'], objects['y'],
                                     3.0, err=bkg.globalrms, gain=1.0)

In [ ]:
# show the first 10 objects results:
for i in range(10):
    print("object {:d}: flux = {:f} +/- {:f}".format(i, flux[i], fluxerr[i]))